In [3]:
%reload_ext autoreload
%autoreload 2

import os
import typing
from typing import Callable, Union, Dict

import pathlib
from pathlib import Path
Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]
import sys
sys.path.append("../wsi_processing_pipeline/")
import tile_extraction
from tile_extraction import tiles, util, slide
import fastai
from fastai.vision.data import ImageList
from fastai.vision.image import pil2tensor
from fastai.basic_train import load_learner
import pandas as pd
import numpy as np


base_path = Path('/home/Deep_Learner/shared/Datasets/Hypophysenadenome/')
wsis_path = base_path/'gonadotrop'

In [2]:
wsi_path = wsis_path/'10-16-Z-HE.ndpi';wsi_path

PosixPath('/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi')

In [122]:
tilesummaries = tiles.WsiOrROIToTilesMultithreaded(wsiPaths=[wsi_path], 
                                                    tilesFolderPath=None, 
                                                    tileHeight=512, 
                                                    tileWidth=512, 
                                                    tile_naming_func=tiles.get_wsi_name_from_path_pituitary_adenoma_entities,
                                                    tile_score_thresh=0.5,
                                                    tileScoringFunction=tiles.scoring_function_1,
                                                    is_wsi=True,
                                                    level=0,
                                                    save_tiles=False,
                                                    return_as_tilesummary_object=True)

Starting to process /home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi


In [123]:
ts = tilesummaries[0]

In [124]:
ts.top_tiles()
tiles.show_wsi_with_marked_tiles(tilesummary=ts, wsi_path=ts.wsi_path)

/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi: Number of tiles that will be kept/all possible tiles: 1030/6734
/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi: Number of tiles that will be kept/all possible tiles: 1030/6734


In [125]:
tiles_df = pd.DataFrame([t.get_name() for t in ts.top_tiles()], columns=['name'])

tile_name_to_tile_object = {}
for t in ts.top_tiles():
    tile_name_to_tile_object[t.get_name()] = t

def open_custom(self, fn):
    "Open image in `fn`."
    return open_image_custom(fn, convert_mode=self.convert_mode, after_open=self.after_open)

def open_image_custom(fn:typing.Union[pathlib.Path, str], 
                      div:bool=True, 
                      convert_mode:str='RGB', 
                      cls:type=fastai.vision.Image, 
                      after_open:Callable=None)->fastai.vision.Image:
        "Open image in `fn`."
        fn = Path(fn)
        tile_name = fn.name
        t = tile_name_to_tile_object[tile_name]
        tile = tiles.ExtractTileFromWSI(path=t.wsi_path, 
                                        x=t.get_x(), 
                                        y=t.get_y(), 
                                        width=t.get_width(), 
                                        height=t.get_height(), 
                                        level=t.level)
        tile = tile.convert(convert_mode)
        if after_open: 
            tile = after_open(tile)
        tile = pil2tensor(tile,np.float32)
        if div: 
            tile.div_(255)
        return cls(tile)
        
fastai.vision.data.ImageList.open = open_custom
fastai.vision.image.open_image = open_image_custom

/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi: Number of tiles that will be kept/all possible tiles: 1030/6734
/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi: Number of tiles that will be kept/all possible tiles: 1030/6734


In [126]:
img_list = ImageList.from_df(df=tiles_df, path='')

In [7]:
export_name = f'n:13-iteration:4-weights:bestmodel_lr2=1e-7--lr3=1e-6_9.pkl'

In [127]:
learner = load_learner(path=Path('./'), file=export_name, test=img_list)

In [141]:
learner.data.batch_size = 8

In [128]:
preds, y = learner.get_preds(ds_type=fastai.basic_data.DatasetType.Test)

In [129]:
thresh = 0.5

In [130]:
preds_bool = (preds > thresh).float()

In [131]:
preds_bool

tensor([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]])

In [132]:
preds_bool.sum(0)

tensor([  0.,   0., 891., 409.])

In [ ]:
probs_wsi = preds_bool.sum(0)/len(preds_bool);probs_wsi

In [ ]:
classes = ['acth', 'silent', 'lh', 'fsh']

In [ ]:
result = {}
for n, c in enumerate(classes):
    result[c] = probs_wsi[n].item()

In [ ]:
result

In [4]:
def predict_pga_entity(path_exported_learner:Union[pathlib.Path, str], wsi_path:Union[pathlib.Path, str])->Dict:
    """
    Arguments:
        path_exported_learner: path to file that has been exported via fastai.basic_train.Learner.export()
        wsi_path: path to a whole-slide image
    Returns:
        returns a dictionary with probabilities for the classes: ['acth', 'silent', 'lh', 'fsh']
    """
    # calculate tiles 
    tilesummaries = tiles.WsiOrROIToTilesMultithreaded(wsiPaths=[wsi_path], 
                                                    tilesFolderPath=None, 
                                                    tileHeight=512, 
                                                    tileWidth=512, 
                                                    tile_naming_func=tiles.get_wsi_name_from_path_pituitary_adenoma_entities,
                                                    tile_score_thresh=0.7,
                                                    tileScoringFunction=tiles.scoring_function_1,
                                                    is_wsi=True,
                                                    level=0,
                                                    save_tiles=False,
                                                    return_as_tilesummary_object=True)
    ts = tilesummaries[0]
    
    
    
    
    #overwrite fastai's function for opening images !!!TODO: change this by using a custom DataLoader Implementation
    def open_custom(self, fn):
        "Open image in `fn`."
        return open_image_custom(fn, convert_mode=self.convert_mode, after_open=self.after_open)

    def open_image_custom(fn:typing.Union[pathlib.Path, str], 
                      div:bool=True, 
                      convert_mode:str='RGB', 
                      cls:type=fastai.vision.Image, 
                      after_open:Callable=None)->fastai.vision.Image:
        "Open image in `fn`."
        fn = Path(fn)
        tile_name = fn.name
        t = tile_name_to_tile_object[tile_name]
        tile = tiles.ExtractTileFromWSI(path=t.wsi_path, 
                                        x=t.get_x(), 
                                        y=t.get_y(), 
                                        width=t.get_width(), 
                                        height=t.get_height(), 
                                        level=t.level)
        tile = tile.convert(convert_mode)
        if after_open: 
            tile = after_open(tile)
        tile = pil2tensor(tile,np.float32)
        if div: 
            tile.div_(255)
        return cls(tile)
        
    fastai.vision.data.ImageList.open = open_custom
    fastai.vision.image.open_image = open_image_custom
    
    #create fastai.vision.data.ImageList
    tiles_df = pd.DataFrame([t.get_name() for t in ts.top_tiles()], columns=['name'])

    tile_name_to_tile_object = {}
    for t in ts.top_tiles():
        tile_name_to_tile_object[t.get_name()] = t
    
    img_list = ImageList.from_df(df=tiles_df, path='')
    
    #init learner
    learner = load_learner(path=Path(path_exported_learner).parent, file=Path(path_exported_learner).name, test=img_list)
    learner.data.batch_size = 6
    
    #make predictions on tiles
    preds, y = learner.get_preds(ds_type=fastai.basic_data.DatasetType.Test)
    
    #calculate probabilities for the whole-slide image
    thresh = 0.5
    preds_bool = (preds > thresh).float()
    probs_wsi = preds_bool.sum(0)/len(preds_bool)
    classes = ['acth', 'silent', 'lh', 'fsh']
    result = {}
    for n, c in enumerate(classes):
        result[c] = probs_wsi[n].item()
    return result

In [8]:
p = Path(os.path.join('.', export_name))

In [9]:
predict_pga_entity(path_exported_learner=p, wsi_path=wsi_path)

Starting to process /home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi
/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi: Number of tiles that will be kept/all possible tiles: 320/6734
/home/Deep_Learner/shared/Datasets/Hypophysenadenome/gonadotrop/10-16-Z-HE.ndpi: Number of tiles that will be kept/all possible tiles: 320/6734


{'acth': 0.0,
 'silent': 0.0,
 'lh': 0.9125000238418579,
 'fsh': 0.4593749940395355}